# Learning Pandas from Start (Part III, Multiple DataFrame)

In [4]:
import pandas as pd
import numpy as np

## Manipulating data from multiple DataFrame

### Merge()

Merge() function is similar like the JOIN statement in SQL.   
<img src="Join.png" width="800px">  
This was specified in the parameter: `how{‘left’, ‘right’, ‘outer’, ‘inner’, ‘cross’}, default ‘inner’`

In [8]:
frame1 = pd.DataFrame( {'id':['ball','pencil','pen','mug','ashtray'], 'price': [12.33,11.44,33.21,13.23,33.62]})
frame1

,id,price
0,ball,12.33
1,pencil,11.44
2,pen,33.21
3,mug,13.23
4,ashtray,33.62


In [14]:
frame2 = pd.DataFrame( {'id':['pencil','pencil','ball','pen','knife'], 'color': ['white','red','red','black','silver']})
frame2

,id,color
0,pencil,white
1,pencil,red
2,ball,red
3,pen,black
4,knife,silver


In [15]:
frame3 = pd.merge(frame1, frame2)
frame3

,id,price,color
0,ball,12.33,red
1,pencil,11.44,white
2,pencil,11.44,red
3,pen,33.21,black


Merge() will find columns with same name from both DataFrame, and use it as key to merge 2 dataframe's data. by default it will use the 'inner' join, keep the intersection of keys.  
The key 'pencil' was happened in frame2 twice, so data from frame1 was duplicated become 2 rows.

In [16]:
frame3 = pd.merge(frame1, frame2, how = 'outer')
frame3

,id,price,color
0,ball,12.33,red
1,pencil,11.44,white
2,pencil,11.44,red
3,pen,33.21,black
4,mug,13.23,NaN
5,ashtray,33.62,NaN
6,knife,NaN,silver


When change to 'outer', all data was kept, only those does not have a data value be filled with 'NaN'.  
in many case we don't have two frames which has the same column label name. We can specify the column which being used as keys to map both table rows.  
`on=...`   to specify common names of columns to be used as key, or specify which level of a multiindex to be used as key  
If does not have common label name, then can specify each dataframe separately:  
`left_on=...`  
`right_on=...`

In [17]:
frame2.columns=['sid','color']
frame2

,sid,color
0,pencil,white
1,pencil,red
2,ball,red
3,pen,black
4,knife,silver


In [18]:
frame3=pd.merge(frame1, frame2, left_on='id',right_on='sid',how='outer')
frame3

,id,price,sid,color
0,ball,12.33,ball,red
1,pencil,11.44,pencil,white
2,pencil,11.44,pencil,red
3,pen,33.21,pen,black
4,mug,13.23,NaN,NaN
5,ashtray,33.62,NaN,NaN
6,NaN,NaN,knife,silver


Sometimes maybe want to use the whole index to be used as key (even multiindex. if multiindex the total level and other dataframe columns must be equal).  
this was sepcified using `left_index=True`, or `right_index=True`.  
There is another function `join()`, which join different dataframes using the row indexes. It is similar to `merge()` with the `left_index=True` and `right_index=True`

### concat()

In [21]:
frame3 = pd.concat([frame1, frame2])
frame3

,id,price,sid,color
0,ball,12.33,NaN,NaN
1,pencil,11.44,NaN,NaN
2,pen,33.21,NaN,NaN
3,mug,13.23,NaN,NaN
4,ashtray,33.62,NaN,NaN
0,NaN,NaN,pencil,white
1,NaN,NaN,pencil,red
2,NaN,NaN,ball,red
3,NaN,NaN,pen,black
4,NaN,NaN,knife,silver


Bu default, concat() will put rows from all frames in the list together, and if column labels are different then add columns also.

In [27]:
frame2.columns=['id','color']
frame3 = pd.concat([frame1, frame2])
frame3

,id,price,color
0,ball,12.33,NaN
1,pencil,11.44,NaN
2,pen,33.21,NaN
3,mug,13.23,NaN
4,ashtray,33.62,NaN
0,pencil,NaN,white
1,pencil,NaN,red
2,ball,NaN,red
3,pen,NaN,black
4,knife,NaN,silver


You can see the index also concated and it have duplicated values. can use reset_index() to change it.

In [28]:
frame3.reset_index(inplace=True,drop=True)
frame3

,id,price,color
0,ball,12.33,NaN
1,pencil,11.44,NaN
2,pen,33.21,NaN
3,mug,13.23,NaN
4,ashtray,33.62,NaN
5,pencil,NaN,white
6,pencil,NaN,red
7,ball,NaN,red
8,pen,NaN,black
9,knife,NaN,silver


It is not necessarily always concat along rows. sometimes we want to concat along columns axis. Here is a case:

In [7]:
df = pd.DataFrame(np.random.rand(4,3), index=[i for i in range(4)], columns = ['A', 'B', 'C'])
df

,A,B,C
0,0.961653,0.352191,0.632339
1,0.378248,0.847311,0.918902
2,0.085244,0.924362,0.955071
3,0.639614,0.106804,0.378722


In [29]:
df1 = pd.DataFrame(np.random.randint(10,20,(5,4)), index=[i for i in range(5)], columns=['W','X','Y','Z'])
df1

,W,X,Y,Z
0,17,18,14,11
1,19,14,18,16
2,14,16,18,12
3,10,18,10,16
4,13,14,16,14


In [30]:
df2 = pd.concat([df,df1],axis=1)
df2

,A,B,C,W,X,Y,Z
0,0.961653,0.352191,0.632339,17,18,14,11
1,0.378248,0.847311,0.918902,19,14,18,16
2,0.085244,0.924362,0.955071,14,16,18,12
3,0.639614,0.106804,0.378722,10,18,10,16
4,NaN,NaN,NaN,13,14,16,14


The concatenation happens along column axis. Index are used as keys to identifiy how to join different rows data. for the rows that does not have data will be filled with `NaN`  
It can also take a `join=...` parameter to decide how to make the join. can be either `inner`, or `outer`

In [31]:
df2 = pd.concat([df,df1],axis=1,join='inner')
df2

,A,B,C,W,X,Y,Z
0,0.961653,0.352191,0.632339,17,18,14,11
1,0.378248,0.847311,0.918902,19,14,18,16
2,0.085244,0.924362,0.955071,14,16,18,12
3,0.639614,0.106804,0.378722,10,18,10,16


There is another function `append()`, it is similar functions as `concat()`, just with different form of how to use it.

### pivot_table()

Pivot is another way of analyze the data inside the dataframe.  
<img src="reshaping_pivot.png" width="600px">  
This is a common operation in microsoft excel to analy

In [41]:
dict_data = {'date':['2000-1-1','2000-1-2','2000-1-3','2000-1-1','2000-1-2','2000-1-3','2000-1-1','2000-1-2','2000-1-3'], 
             'variable':['A','A','A','B','B','B','C','C','C'],
             'value':np.random.rand(9)}
dfa=pd.DataFrame(dict_data)
dfa

,date,variable,value
0,2000-1-1,A,0.047951
1,2000-1-2,A,0.843412
2,2000-1-3,A,0.108025
3,2000-1-1,B,0.427091
4,2000-1-2,B,0.989131
5,2000-1-3,B,0.250369
6,2000-1-1,C,0.632294
7,2000-1-2,C,0.516561
8,2000-1-3,C,0.508207


In [42]:
dfa.dtypes

date         object
variable     object
value       float64
dtype: object

In [43]:
dfp=pd.pivot_table(dfa,index="date", columns="variable", values="value")
dfp

variable,A,B,C
date,,,
2000-1-1,0.047951,0.427091,0.632294
2000-1-2,0.843412,0.989131,0.516561
2000-1-3,0.108025,0.250369,0.508207


In [44]:
dict_data = {'date':['2000-1-1','2000-1-2','2000-1-3','2000-1-1','2000-1-2','2000-1-3','2000-1-1','2000-1-2','2000-1-3'], 
             'variable':['A','A','A','B','B','B','C','C','C'],
             'value':np.random.rand(9)}
dfb=pd.DataFrame(dict_data)
dfc=dfa.append(dfb)
dfc

,date,variable,value
0,2000-1-1,A,0.047951
1,2000-1-2,A,0.843412
2,2000-1-3,A,0.108025
3,2000-1-1,B,0.427091
4,2000-1-2,B,0.989131
5,2000-1-3,B,0.250369
6,2000-1-1,C,0.632294
7,2000-1-2,C,0.516561
8,2000-1-3,C,0.508207
0,2000-1-1,A,0.917971


In [47]:
dfp=pd.pivot_table(dfc,index="date", columns="variable", values="value",aggfunc='count')
dfp

variable,A,B,C
date,,,
2000-1-1,2,2,2
2000-1-2,2,2,2
2000-1-3,2,2,2


In [50]:
dfp=pd.pivot_table(dfc,index="date", columns="variable", values="value",aggfunc=['mean','sum'])
dfp

mean                           sum                    
variable         A         B         C         A         B         C
date                                                                
2000-1-1  0.482961  0.533435  0.530615  0.965922  1.066869  1.061230
2000-1-2  0.680162  0.834048  0.607875  1.360324  1.668097  1.215750
2000-1-3  0.243394  0.531495  0.660663  0.486788  1.062989  1.321326